In [2]:
#packages
import pandas as pd
import numpy as np
import warnings
import sqlite3

from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBRFRegressor

In [3]:

# hide warnings
warnings.filterwarnings('ignore')

In [18]:
def create_connection(path):
    connection = None 
    connection = sqlite3.connect(path)
    connection.text_factory = str

    return connection

In [28]:
def execute_query(connection, query):
    cursor = connection.cursor()
    if query == "":
        return "Query Blank"
    else:
        cursor.execute(query)
        connection.commit()
        return "Query executed successfully"

In [13]:

def create_table(connection, tableName, columns):
    sql = F"DROP TABLE IF EXISTS {tableName}; "
    execute_query(connection, sql)
    sql = F"CREATE TABLE {tableName} ({columns}); "
    return execute_query(connection, sql)


In [14]:

def insert_into_table(connection, table_name, columns, records):
    values = ""
    for record in records:
        record = record.replace("'", "")
        if len(values) > 0:
            values+=","
        if str.isnumeric(record):
            values+= str(record)
        else:
            values +=F"'{record}'"

    sql= F"INSERT INTO {table_name} ({columns}) VALUES ({values}); "
    return execute_query(connection, sql)


In [26]:
import csv

def is_successful(self, result):
    return "success" in result
    
def import_file(connection, file, table_name, columns):

    with open(file, encoding="utf8") as file:

        lines = csv.reader(file, delimiter=',')
        for line in lines:
                            
            result = insert_into_table(connection, table_name, columns, line)
            if not is_successful(result):
                return result

    return result

In [20]:
con = create_connection('C:\\Data\\fire_data_v4.db\\fire_data_v4.db')


In [30]:
create_table(con, "Truck_Engine_Fire_Station", "Station text, Type text, Unit text, FireStation text, Facility_ID text")


'Query executed successfully'

In [31]:
import_file(con, "C:\\repos\\gatech\\datavisualanalytics\\firstrespondersites\\jyEDA\\clean\\truck_engine_fire_station.csv", "truck_engine_fire_station", ["Station", "Type", "Unit", "Fire Station", "Facility_ID"])


OperationalError: table truck_engine_fire_station has no column named 'Station', 'Type', 'Unit', 'Fire Station', 'Facility_ID'